# Fundamentals of Software Systems (FSS)
**Software Evolution – Part 02 Assignment**

## Submission Guidelines

To correctly complete this assignment you must:
* Carry out the assignment in a team of 2 to 4 students.
* Carry out the assignment with your team only. You are allowed to discuss solutions with other teams, but each team should come up its own personal solution. A strict plagiarism policy is going to be applied to all the artifacts submitted for evaluation.
* As your submission, upload the filled Jupyter Notebook (including outputs) together with the d3 visualization web pages (i.e. upload everything you downloaded including the filled Jupyter Notebook plus your `output.json`)
* The files must be uploaded to OLAT as a single ZIP (`.zip`) file by Dec 15, 2025 @ 23:55.


## Group Members
* Firstname, Lastname, Immatrikulation Number
* **TO BE FILLED**

## Task Context

In this assigment we will be analyzing the _elasticsearch_ project. All following tasks should be done with the subset of commits from tag `v1.0.0` to tag `v1.1.0`.

In [ ]:
from enum import Enum
import pydriller

class Modification(Enum):
    ADDED = "Lines added"
    REMOVED = "Lines removed"
    TOTAL = "Lines added + lines removed"
    DIFF = "Lines added - lines removed"


repo = pydriller.Repository("/Users/dambrosm/elastic/source/elasticsearch",
                            from_tag="v1.0.0",
                            to_tag="v1.1.0")


## Task 1: Author analysis

In the following, please consider only `java` files.

The first task is to get an overview of the author ownership of the _elasticsearch_ project. In particular, we want to understand who are the main authors in the system between the two considered tags, the authors distribution among files and the files distribution among authors. To this aim, perform the following:
- create a dictionary (or a list of tuples) with the pairs author => number of modified files
- create a dictionary (or a list of tuples) with the pairs file => number of authors who modified the file
- visualize the distribution of authors among files: the visualization should have on the x axis the number of authors per file (from 1 to max), and on the y axis the number of files with the given number of authors (so for example the first bar represent the number of files with single author)
- visualize the distribution of files among authors: the visualization should have on the x axis the number of files per author (from 1 to max), and on the y axis the number of authors that own the given number of files (so for example the first bar represent the minor contributors, i.e., the number of authors who own 1 file)

Comment the two distribution visualizations.



Now, let's look at the following 3 packages in more details:
1. `src/main/java/org/elasticsearch/search`
2. `src/main/java/org/elasticsearch/index`
3. `src/main/java/org/elasticsearch/action`

Create a function that, given the path of a package and a modification type (see class Modification above), returns a dictionary of authors => number, where the number counts the total lines added or removed or added+removed or added-removed (depending on the given Modification parameter), for the given package. To compute the value at the package level, you should aggregate the data per file.

Using the function defined above, visualize the author contributions (lines added + lines removed). The visualization should have the author on the x axis, and the total lines on the y axis. Sort the visualization in decreasing amount of contributions, i.e., the main author should be the first.

Compare the visualization for the 3 packages and comment.

## Task 2: Knowledge loss

We now want to analyze the knowledge loss when the main contributor of the analyzed project would leave. For this we will use the circle packaging layout introduced in the "Code as a Crime Scene" book. This assignment includes the necessary `knowledge_loss.html` file as well as the `d3` folder for all dependencies. You task is to create the `output.json` file according to the specification below. This file can then be visualized with the files provided.

For showing the visualization, once you have the output as `output.json` you should
* make sure to have the `knowledge_loss.html` file in the same folder
* start a local HTTP server in the same folder (e.g. with python `python3 -m http.server`) to serve the html file (necessary for d3 to work)
* open the served `knowledge_loss.html` and look at the visualization

For the package you identify as the worst in terms of knowledge loss, investigate the author contributions using the function defined in the previous exercise and comment how the situation is, e.g. how big the gap between the main author and the second biggest contributor for the selected package is.

### Output Format for Visualization

* `root` is always the root of the tree
* `size` should be the total number of lines of contribution
* `weight` can be set to the same as `size`
* `ownership` should be set to the percentage of contributions from the main author (e.g. 0.98 for 98% if contributions coming from the main author)

```
{
  "name": "root",
  "children": [
    {
      "name": "test",
      "children": [
        {
          "name": "benchmarking",
          "children": [
            {
              "author_color": "red",
              "size": "4005",
              "name": "t6726-patmat-analysis.scala",
              "weight": 1.0,
              "ownership": 0.9,
              "children": []
            },
            {
              "author_color": "red",
              "size": "55",
              "name": "TreeSetIterator.scala",
              "weight": 0.88,
              "ownership": 0.9,
              "children": []
            }
          ]
        }
      ]
    }
  ]
}
```

### JSON Export

For exporting the data to JSON you can use the following snippet:

```
import json

with open("output.json", "w") as file:
    json.dump(tree, file, indent=4)
```

In [2]:
import pydriller
from  collections import Counter

PATH = "/Users/colin/Documents/Studium/SoSy/elasticsearch"

def compute_ownership(path):
    repo = pydriller.Repository(path, from_tag="v0.90.0", to_tag="v1.0.0.RC1")
    file_dict = {}
    count = 0

    for commit in repo.traverse_commits():
        count += 1
        if count % 250 == 0:
            print(f"Processed {count} commits so far")

        author = commit.author
        files = commit.modified_files
        for file in files:
            if file.filename in file_dict:
                file_dict[file.filename].update([author.name])  
            else:
                file_dict[file.filename] = Counter()
                file_dict[file.filename].update([author.name])
    
    print(f"Processed {count} commits")
    
    ownership_dict = {}
    for filename, file in file_dict.items():
        top_author, top_count = file.most_common(1)[0]
        total = file.total()
        percentage = top_count/total
        ownership_dict[filename] = (top_author, percentage)

    return ownership_dict

ownership_dict = compute_ownership(PATH)

Processed 250 commits so far
Processed 500 commits so far
Processed 750 commits so far
Processed 1000 commits so far
Processed 1250 commits so far
Processed 1500 commits so far
Processed 1750 commits so far
Processed 1934 commits


In [3]:
import os
import json
import hashlib
import time

PATH = "/Users/colin/Documents/Studium/SoSy/elasticsearch"

def generate_color(input_str):
    """ generate a random but consistent color based on an input """

    def stable_hash(s):
        return int(hashlib.sha256(s.encode()).hexdigest(), 16)

    if len(input_str) == 1:
        r = abs(stable_hash(input_str)) % 256
        return f"rgb({r},0,0)"
    
    if len(input_str) == 2:
        r = abs(stable_hash(input_str[0])) % 256
        g = abs(stable_hash(input_str[1])) % 256
        return f"rgb({r},{g},0)"
    
    split = len(input_str) // 3
    r = abs(stable_hash(input_str[:split])) % 256
    g = abs(stable_hash(input_str[split:2*split])) % 256 
    b = abs(stable_hash(input_str[2*split:])) % 256 
    
    return f"rgb({r},{g},{b})"

def count_lines(file_path):
    with open(file_path, 'r', errors='ignore') as f:
        return len(f.readlines())

def generate_json_tree(path, ownership_dict, tree=None):
    if tree is None:
        tree = {
            "name": "root",
            "children":[],
        }

    contents = os.listdir(path)
    for item in contents:
        full_path = os.path.join(path, item)
        if os.path.isfile(full_path):
            # Files that weren't edited can't be assessed on ownership
            if item not in ownership_dict:
                continue

            lines = count_lines(full_path)
            entry = {
                "author_color": generate_color(ownership_dict[item][0]),
                "size": lines,
                "name": item,
                "weight": lines,
                "ownership": ownership_dict[item][1],
                "children": [],
            }

            tree["children"].append(entry)
        
        else:
            entry = {
                "name": item,
                "children": [],
            }
            entry = generate_json_tree(full_path, ownership_dict, tree=entry)

            # ignore empty branches, not relevant for the visualization
            if len(entry["children"]) > 0:
                tree["children"].append(entry)
    
    return tree

tree = generate_json_tree(PATH, ownership_dict=ownership_dict)

with open("output.json", "w") as file:
    json.dump(tree, file, indent=4)

## Task 3: Code Churn Analysis

The third and last task is to analyze the code churn of the _elasticsearch_ project. For this analysis we look at the code churn, meaning the daily change in the total number of lines of the project. Visualize the code churn over time bucketing the data by day. Remember that you'll need to interpolate the data for days when there are no commits. Chose an interpolation strategy and justify it.

Look at the churn trend over time and identify two outliers. For each of them:
- identify if it was caused by a single or multiple commits (since you are bucketing the data by day)
- find the hash of the involved commit(s)
- find the involved files
- look at the actual diff

Based on the above, discuss if the outlier is a false positive or should be a reason for concern.